# langchain: un esempio di Q&A sul contenuto di un documento

Il problema che si vuole risolvere qui è di usare un LLM per fare domande in linguaggio naturale a proposito del contenuto di uno o più documenti che non sono stati letti durante il pre-training.

Se il documento non è sufficientemente breve, e quindi non può essere interamente contenuto in un prompt insieme con la domanda, è necessario preliminarmente dividerlo in parti, identificare quali parti sono rilevanti per rispondere alla domanda, e passare nel prompt solo queste. Per realizzare questo processo preliminare occorre essere in grado di effettuare una ricerca semantica sul documento, e per questo occorre:
* dividere il documento in parti (_chunks_);
* fare l'_embedding_ di ogni parte;
* scrivere gli _embeddings_ in un database vettoriale.
Quando questo è stato fatto, data la domanda:
* si fa una ricerca di similarità nel database,
* si estraggono le parti più rilevanti, e
* le si introduce nel prompt insieme con la domanda stessa,
inviando infine il prompt all'oggetto `load_qa_chain`.

[virtenv `langchain`: langchain, openai, chroma]

In [4]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

# leggi il documento, non abbastanza breve
loader = TextLoader('./8tuple_l.txt', encoding='utf8')
data = loader.load()

# dividi il documento in parti
text_splitter = CharacterTextSplitter(
    separator='\n',
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
chunks = text_splitter.split_text(data[0].page_content)
#print(len(chunks))

# crea l'embedding
embeddings = OpenAIEmbeddings()
vector_db = Chroma.from_texts(chunks, embeddings)

# crea la domanda
query = "What is the fourth element of the 8-tuple and what is its purpose?"

# cerca le parti corrispondenti alla domanda
relevant_chunks = vector_db.similarity_search(query, k=8)
#print(len(relevant_chunks))

# attiva l'llm
llm = OpenAI(temperature=0)

# invia la domanda con il contesto, ottieni e visualizza la risposta
chain = load_qa_chain(llm, chain_type='stuff')
print(chain.run(input_documents=relevant_chunks, question=query).strip())

Using embedded DuckDB without persistence: data will be transient


The fourth element of the 8-tuple is Ω and its purpose is to describe the environment of the system, including the model user, on the system.


Come si vede, il risultato è di qualità media. Il motivo pare sia la limitata qualità della ricerca semantica, che non restituisce le parti effettivamente rilevanti del documento.